<table style="margin: auto; background-color: white;">
    <tr>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1R6PphT9jmd2vikODFPf6cW54QtZ29o2a' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1R6PphT9jmd2vikODFPf6cW54QtZ29o2a' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1R6PphT9jmd2vikODFPf6cW54QtZ29o2a' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
    </tr>
</table>

# TUTORIAL 4 - FMNIST E SELEÇÃO DE CLIENTES

Bem-vindo! Neste tutorial você aprenderá sobre a interface de programação da plataforma **Flautim** e também como montar um experimento simples de classificação usando o dataset [FMNIST](https://huggingface.co/datasets/zalando-datasets/fashion_mnist) com **seleção de clientes**. 

É recomendado que você já esteja familiarizado com aprendizado federado e utilização da plataforma Flautim, tendo realizado algum dos outros tutoriais previamente.

O código desse tutorial pode ser acessado em: [clique aqui](./TUTORIAL_4/).


Vamos começar entendendo a interface de programação da **Flautim**, representada na figura abaixo. A **Flautim_api** é uma biblioteca modularizada que facilita a realização de experimentos de aprendizado de máquina, seja convencional/centralizado ou federado.

Todo projeto **Flautim** precisa herdar essa biblioteca, que contém submódulos específicos para diferentes tecnologias (por exemplo, submódulos para PyTorch, TensorFlow, etc). Neste tutorial usaremos o submódulo para PyTorch.

<div style="text-align: center;"> <table style="margin: auto;"> <tr> <td> <img src='https://drive.google.com/uc?export=view&id=1QOI4jWrwS979xhW_wlGzkPa2bMA-giuc' alt="Interface da plataforma Flautim" width="800" /> </td> </tr> </table> </div>


Dentro de cada submódulo existem três componentes principais (classes):

**1. Dataset:** é utilizado para representar os dados do experimento. Esta classe pode ser reutilizada em diversos experimentos e com diferentes modelos, sendo o componente mais versátil e reutilizável. Os usuários podem importar os dados de diversas fontes, como arquivos locais ou bases de dados online, desde que a classe Dataset seja herdada.

**2. Model:** representa qualquer conjunto de parâmetros treináveis dentro do projeto. Ela permite a aplicação de técnicas de aprendizado de máquina por meio de treinamento desses parâmetros. No caso de PyTorch, a classe herda a nn.Module, que define a estrutura e os parâmetros treináveis do modelo.

**3. Experiment:** define o ciclo de treinamento e validação. Existem dois tipos principais de experimentos: o experimento centralizado, que segue o fluxo
convencional de aprendizado de máquina, e o experimento federado, adaptado para
aprendizado federado. Esta classe inclui duas funções principais, um loop de
treinamento e um loop de validação, que realizam a atualização dos parâmetros e
cálculo das métricas de custo, respectivamente.

Além desses três componentes principais, há também um módulo chamado Common. Este módulo fornece acesso a classes essenciais para o gerenciamento de dados e monitoramento do treinamento.


Com essa visão geral, você está pronto para começar montar seus próprios experimentos. Vamos ao passo a passo!

### Passo 1: Criando o dataset que será usado no experimento

Um conjunto de dados no Flautim é acessado por um arquivo .py que deve conter uma classe que herda de Dataset.

**Exemplo: Implementando a Classe FMNISTDataset**

O código abaixo implementa uma classe FMNISTDataset utilizando o dataset FMNIST obtido pelo Hugging Face para resolver um problema de classificação.

In [ ]:
from flautim.pytorch.Dataset import Dataset
import numpy as np
import pandas as pd
import torch
import copy
from torch.utils.data import DataLoader
    
class FMNISTDataset(Dataset):

    def __init__(self, FM_Normalization, EVAL_Transforms, TRAIN_Transforms, partition, **kwargs):
    
        name = kwargs.get('name', 'FMNIST')
    
        super(FMNISTDataset, self).__init__(name, **kwargs)
        
        self.FM_Normalization = FM_Normalization
        self.EVAL_Transforms = EVAL_Transforms
        self.TRAIN_Transforms = TRAIN_Transforms
        
        self.feature_name = kwargs.get("feature_name", 'image')
        self.split = kwargs.get("split_data", True)
        
        if self.split:
            partition = partition.train_test_split(test_size=0.2, seed=42)

        self.train_partition = partition["train"]
        self.test_partition = partition["test"]
        
    def dataloader(self, validation = False):
        tmp = self.validation() if validation else self.train()
        return DataLoader(tmp, batch_size = self.batch_size, num_workers = 1)
    
        
    
    def apply_train_transforms(self, batch):
        """Apply transforms to the partition from FederatedDataset."""
        batch[self.feature_name] = [self.TRAIN_Transforms(img) for img in batch[self.feature_name]]
        return batch


    def  apply_eval_transforms(self, batch):
        """Apply transforms to the partition from FederatedDataset."""
        batch[self.feature_name] = [self.EVAL_Transforms(img) for img in batch[self.feature_name]]
        return batch


    def train(self):

        return self.train_partition.with_transform(self.apply_train_transforms)


    def validation(self):
        
        return self.test_partition.with_transform(self.apply_eval_transforms)
 

### Passo 2: Criando o modelo que será usado no experimento

Agora, vamos criar a classe que implementa o modelo. Essa classe deve herdar da classe Model.


**Exemplo: Implementando a Classe PoCModel_2HiddenLayers**

A classe ```PoCModel_2HiddenLayers``` implementa uma rede neural convolucional baseada na rede proposta do Paper Power-Of-Choice, com as seguintes camadas:
* Pre-processamento (flatten): a entrada (assumidamente um lote de imagens 28x28) é achatada em um vetor de 784 características
* Uma camada oculta totalmente conectada, com entrada de 784 neurônios e saída de 256 neurônios. Ativação ReLU.
* Uma camada oculta totalmente conectada, com entrada de 256 neurônios e saída de 128 neurônios. Ativação ReLU.
* Uma camada de saída totalmente conectada com número de neurônios igual ao número de classes do problema.

Essa classe pode ser incluída, por exemplo, em um arquivo MNISPoCModel_2HiddenLayers.py

In [ ]:
from flautim.pytorch.Model import Model
import torch
import torch.nn as nn
import torch.nn.functional as F

class PoCModel_2HiddenLayers(Model):
    """
    MLP Profundo com DUAS CAMADAS OCULTAS para FMNIST (784 -> 256 -> 128 -> 10)
    Modelo como descrito no paper de referência de Power of Choice
    """
    def __init__(self, context, num_classes: int, **kwargs) -> None:
        super(PoCModel_2HiddenLayers, self).__init__(context, name = "2HiddenLayersNN", version = 1, id = 1, **kwargs)

        self.hidden1 = nn.Linear(784, 256)
        self.hidden2 = nn.Linear(256, 128)
        
        self.output_layer = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1) # Flatten da imagem (28x28 -> 784)
        
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        
        x = self.output_layer(x)
        return x

### Passo 3: Criando o experimento

Por fim, será criado o experimento, isto é, uma classe que implementa os loops de treinamento e validação do modelo PoCModel_2HiddenLayers no dataset FMNISTDataset. Para isso, precisamos criar dois arquivos .py, o run.py (que deve ter obrigatoriamente esse nome) e o .py responsável por implementar o experimento, descritos a seguir:

**1. Arquivo run.py:**

* Esse arquivo é o ponto de entrada de todo experimento Flautim, pois é ele
que deve iniciar a classe do experimento e também um modelo e um Dataset.

**2. Arquivo .py do experimento:**

* Esse arquivo deve conter uma classe que implemente os métodos de treinamento (training_loop) e validação (evaluation_loop) do modelo. Essa classe deve herdar da classe Experiment.

Esse tutorial cobrirá dois tipos de experimentos, um experimento centralizado e outro descentralizado. Portanto, o passo 3 será dividido entre esses dois cenários.

#### **Seleção de Clientes**
Ao considerar o funcionamento do aprendizado federado, onde os modelos locais dos clientes são enviados para o servidor central que utilizará uma estratégia de agregação (como FedAvg) para treinar o modelo global, a participação dos clientes é um fator importante para o desempenho do modelo. 

Ao utilizarmos, por exemplo, o [framework flower](https://flower.ai/docs/framework/#) sem definir uma seleção específica, a quantidade percentual de clientes definida no ```fraction_fit``` será usada para escolher **aleatoriamente** entre os clientes disponíveis na rodada. 
Essa entre os clientes é feita pela  ``` class SimpleClientManager (ClientManager) ```
em ``` (207) sampled_cids = random.sample(available_cids, num_clients) ``` 
e pode ser estudada em https://github.com/adap/flower/blob/main/framework/py/flwr/server/client_manager.py.

##### Definindo uma nova estratégia

Na literatura diversos autores já abordaram o tema, como [Cho et al.](https://proceedings.mlr.press/v151/jee-cho22a/jee-cho22a.pdf) que propõem Power-Of-Choice (POC), uma estrutura de seleção de clientes que direciona a seleção de clientes para clientes com maior perda local, permitindo uma convergência mais rápida e aumentamdo a eficiência da comunicação.

Para definirmos uma nova estratégia de seleção de clientes, usualmente estamos interresados em escolher de acordo com uma informação do treinamento atual (por exemplo a _loss_ para POC). Essas informações só estão disponíveis durante a etapa de treinamento (onde o metodo _fit_ é chamado), anterior a agregação do servidor. Assim, a solução adotada é definir uma nova estratégia extendendo uma existente (como FedAvg) e modificando o método ```configure_fit```.



#### Passo 3.1: Experimento federado
**Implementando a Classe FMNISTExperiment**

No código abaixo, criamos a classe FMNISTExperiment no modo federado com seus métodos training_loop e evaluation_loop para treinar e testar a rede neural. Esses métodos retornam o valor da função de perda e a acurácia de treinamento e de validação.

Repare que o método ```fit``` é redefinido para ter um tratamento extra da _loss_ local a ser utilizado na estratégia POC.

In [ ]:
from flautim.pytorch.federated.Experiment import Experiment
import flautim as fl
import flautim.metrics as flm
import numpy as np
import torch
import time

from collections import OrderedDict
from sklearn.metrics import f1_score

from math import inf

from random import random

# Two auxhiliary functions to set and extract parameters of a model
def set_params(model, parameters):
    """Replace model parameters with those passed as `parameters`."""

    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.from_numpy(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)


def get_params(model):
    """Extract model parameters as a list of NumPy arrays."""
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

class PoCExperimentFMNIST(Experiment):
    """
    Experimento baseline para o MNIST
    Baseado no tutorial do flautim e definições prévias.
    Utiliza o modelo BaselineModelMNIST e o dataset MNISTDataset (do tutorial).
    """

    def __init__(self, model, dataset, context, **kwargs):
        super(PoCExperimentFMNIST, self).__init__(model, dataset, context, **kwargs)
        
        self.criterion = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.01, momentum=0.9)
        self.epochs = kwargs.get('epochs', 1)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.last_loss = inf
        self.last_participation_round = 0
        self.model = model
        self.data_size = len(dataset.train_partition) # Store the size of the local training data

    # --------------------------------
    # Ref: https://discuss.flower.ai/t/custom-client-selection-strategy/63
    def fit(self, parameters, config):
        set_params(self.model, parameters)
        epochs = config.get("epochs", self.epochs) # Get the number of epochs from config
        if epochs == -1:
            return parameters, 0, {"data_size": self.data_size}  # Just return client's data size

        if epochs == 0:
            # Estimate local loss without training the model
            local_loss, _ = self.validation_loop(self.dataset.dataloader(validation=True))
            local_loss += np.random.uniform(low=1e-10, high=1e-9) # Make sure that potential ties are broken at random
            return parameters, 0, {"local_loss": local_loss}  # Return the estimated local loss without updating the parameters

        # Train the model and return the updated parameters
        # self.fit(self.x_train, self.y_train, validation_data=(self.x_test, self.y_test), epochs = self.epochs, verbose=0)
        loss, metrics = self.training_loop(self.dataset.dataloader())
        return get_params(self.model), self.data_size, metrics
    # --------------------------------

    def training_loop(self, data_loader):
        """This method trains the model using the parameters sent by the
        server on the dataset of this client. At then end, the parameters
        of the locally trained model are communicated back to the server"""
        
        self.model.to(self.device)
        self.model.train()
        
        correct, running_loss = 0.0, 0.0

        criterion = torch.nn.CrossEntropyLoss()
        self.model.to(self.device)
        self.model.train()
        for batch in data_loader:
            images, labels = batch["image"].to(self.device), batch["label"].to(self.device)
            self.optimizer.zero_grad()
        
            outputs = self.model(images.to(self.device))
            # todo alterar caso FedProx!
            loss = criterion(self.model(images), labels)
            
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()
            correct += (torch.max(outputs, 1)[1].cpu() == labels.cpu()).sum().item()

        accuracy = correct / len(data_loader.dataset)    
        avg_trainloss = running_loss / len(data_loader)
        
        self.last_loss = avg_trainloss           
        
        return float(avg_trainloss), {'ACCURACY': accuracy}


    def validation_loop(self, data_loader):
        # Usa função evaluate do colab como base
        """Evaluate the model sent by the server on this client's
        local validation set. Then return performance metrics."""

        all_predictions = []
        all_labels = []

        criterion = torch.nn.CrossEntropyLoss()
        correct, loss = 0, 0.0

        self.model.to(self.device)

        self.model.eval()
        with torch.no_grad():
            for batch in data_loader:
                images, labels = batch["image"].to(self.device), batch["label"].to(self.device)
                outputs = self.model(images)
                loss += criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == labels).sum().item()

                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = correct / len(data_loader.dataset)
        f1 = f1_score(all_labels, all_predictions, average='macro')

        return float(loss), {"accuracy": accuracy, "f1": f1}
    

**Implementando o run.py para realização de um experimento federado**

**1. Upload do Conjunto de Dados:**

* *Arquivo Local:* Se o seu conjunto de dados for um arquivo (por exemplo, CSV, NPZ, etc.), faça o upload para a plataforma e carregue-o usando o caminho ./data/nomedoarquivo.

* *URL:* Se o conjunto de dados estiver disponível em uma URL, inclua a URL no seu código e carregue-o diretamente.

**2. Separação dos dados por cliente:**

* Para simular 4 clientes, divida os dados em 4 partes.

**3. Crie uma instância para FMNISTDataset, PoCModel_2HiddenLayers, PoCExperimentFMNIST.**

**4. Execute as funções:**
* ***generate_server_fn:*** Cria a estratégia para o aprendizado federado
* ***generate_client_fn:*** Gera o modelo e o dataset de cada cliente.
* ***evaluate_fn:*** Avalia o modelo global usando o dataset de um dos clientes.
* ***run_federated:*** Executa o experimento federado.


#### **Modificações para seleção de clientes**
É necessário definir uma classe que herde de uma _strategy_ para ser utilizada ao criar o servidor. 

No exemplo abaixo a ``class MyStrategy(FedAvg)`` poderia ser definida em outro arquivo, por exemplo MyStrategy.py, mas por simplicidade foi declarada no run para evitar imports.

Ao definirmos o servidor utilizamos ``strategy = MyStrategy(...)`` e todo o tratamento da seleção de cliente será feito pela mesma. Nesse exemplo o funcionamento base é:
1. Obtém o número de clientes disponíveis
2. Calcula a probabilidade de escolher determinado cliente baseado no tamanho do dataset
3. Seleciona os d (30) clientes com maiores probabilidades como candidatos
4. Dentre estes candidatos, solicita os valores de perdas locais
5. Selecione os m (min_fit_clients = 3) com maiores perdas locais e informa para o ClientManager

In [ ]:
from flautim.pytorch.common import run_federated, weighted_average
from flautim.pytorch import Model, Dataset
from flautim.pytorch.federated import Experiment
import FMNISTDataset, PoCModel_2HiddenLayers, PoCExperimentFMNIST # Alterado!
from PoCExperimentFMNIST import get_params # Alterado!
import flautim as fl
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import DirichletPartitioner
from flwr.common import Context, ndarrays_to_parameters
import flwr
import pandas as pd
import numpy as np
from flwr.server import ServerConfig, ServerAppComponents
from datasets import load_dataset

from random import random
from flwr.server.client_manager import ClientManager
from flwr.common import Parameters, FitIns
from flwr.server.strategy import FedAvg
from flwr.server.client_proxy import ClientProxy
from time import sleep

NUM_PARTITIONS = 100 # Alterado! Numero de clientes totais e numero de partições de dados
DATASET = "ylecun/mnist" # Alterado!

from torchvision.transforms import (
    Compose,
    Normalize,
    RandomCrop,
    RandomHorizontalFlip,
    ToTensor,
)


FM_NORMALIZATION = ((0.1307,), (0.3081,))
EVAL_TRANSFORMS = Compose([ToTensor(), Normalize(*FM_NORMALIZATION)])
TRAIN_TRANSFORMS = Compose(
    [
        RandomCrop(28, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(*FM_NORMALIZATION),
    ]
)

# -------------------------------
# Ref:https://discuss.flower.ai/t/custom-client-selection-strategy/63
class MyStrategy(FedAvg):
    """Behaves just like FedAvg but with a modified sampling.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.p = [] # probabilities for each client based on data size?
        self.d = 30 # number of candidate clients to consider each round
        self.m = self.min_fit_clients # number of clients to select each round
        # self.prev_clients = [] # store client proxies from previous round
    
    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager = ClientManager # client_manager alterado!
        ) -> list[tuple[ClientProxy, FitIns]]: 
        """Configure the next round of training."""
        # Get list with all the available clients (K clients)
        available_clients = list(client_manager.clients.values())

        print(f"Round {server_round} - Available clients: {[client.cid for client in available_clients]}")
        
        if self.p == []:
            data_sizes = {
                client.cid: client.fit(
                    FitIns(parameters, {"epochs": -1}), 
                    timeout=60, 
                    group_id=str(client.cid)
                ).metrics.get("data_size", 0)
                for client in available_clients
            }
            
            for key, value in data_sizes.items():
                print(f"Cliente {key} tem {value} data_size.")

            # Compute selection probabilities based on data size
            total_size = sum(data_sizes.values())
            self.p = [size / total_size for size in data_sizes.values()]

            for i in self.p:
                print(f"p = {i}")

            p_np = np.array(self.p)
            soma_real = p_np.sum()
            
            p_np /= soma_real # Normaliza a maior parte do array
            last_index = len(p_np) - 1
            desvio = 1.0 - p_np[:-1].sum()
            p_np[last_index] = desvio # Força o último elemento a compensar o desvio
            self.p = p_np.tolist()

        # Sample d clients with a probability proportional to their data size
        candidate_clients = np.random.choice(
            available_clients,
            size=min(self.d, len(available_clients)),
            p=self.p,
            replace=False
        )

        # Request the candidate clients to compute their local losses and return them to the server
        local_losses = {
            client.cid: client.fit(
                FitIns(parameters, {"epochs": 0}), 
                timeout=4,
                group_id=str(client.cid) #? Added to fix the error
            ).metrics.get('local_loss', float('inf'))
            for client in candidate_clients
        }

        # Select the top m clients with the highest local losses
        selected_clients_cids = sorted(local_losses, key=local_losses.get, reverse=True)[:self.m]
        selected_clients = []
        selected_clients.append([key for key in selected_clients_cids])

        # Return the selected clients with the FitIns objects
        return [(client_manager.clients.get(cid), FitIns(parameters, {})) for cid in selected_clients_cids]
        

# -------------------------------


def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
    }
    return config



def generate_server_fn(context, eval_fn, **kwargs):

    def create_server_fn(context_flwr:  Context):

        net = PoCModel_2HiddenLayers.PoCModel_2HiddenLayers(context, num_classes = 10, suffix = 0)
        ndarrays = get_params(net)
        global_model_init = ndarrays_to_parameters(ndarrays)

        strategy = MyStrategy(
                          evaluate_fn=eval_fn,
                          on_fit_config_fn = fit_config,
                          on_evaluate_config_fn = fit_config,
                          evaluate_metrics_aggregation_fn=weighted_average,  # callback defined earlier
                          initial_parameters=global_model_init,  # initialised global model,
                          fraction_fit=0.03,
                          min_fit_clients=3,
                        )
        num_rounds = 100
        config = ServerConfig(num_rounds=num_rounds)

        return ServerAppComponents(config=config, strategy=strategy)
    return create_server_fn

def generate_client_fn(context):

    def create_client_fn(context_flwr:  Context):

        global fds

        cid = int(context_flwr.node_config["partition-id"])

        partition = fds.load_partition(cid)

        model = PoCModel_2HiddenLayers.PoCModel_2HiddenLayers(context, num_classes = 10, suffix = cid)

        dataset = FMNISTDataset.FMNISTDataset(FM_NORMALIZATION, EVAL_TRANSFORMS, TRAIN_TRANSFORMS, partition, batch_size = 32, shuffle = False, num_workers = 0)

        return PoCExperimentFMNIST.PoCExperimentFMNIST(model, dataset,  context).to_client()

    return create_client_fn


def evaluate_fn(context):
    def fn(server_round, parameters, config):
        """This function is executed by the strategy it will instantiate
        a model and replace its parameters with those from the global model.
        The, the model will be evaluate on the test set (recall this is the
        whole MNIST test set)."""

        global FM_NORMALIZATION, EVAL_TRANSFORMS, TRAIN_TRANSFORMS, DATASET
        global fds

        model = PoCModel_2HiddenLayers.PoCModel_2HiddenLayers(context, num_classes = 10, suffix = "FL-Global")
        model.set_parameters(parameters)

        partition = fds.load_partition(0)

        dataset = FMNISTDataset.FMNISTDataset(FM_NORMALIZATION, EVAL_TRANSFORMS, TRAIN_TRANSFORMS, partition, batch_size = 32, shuffle = False, num_workers = 0)
        dataset.test_partition = load_dataset(DATASET)["test"]

        experiment = PoCExperimentFMNIST.PoCExperimentFMNIST(model, dataset, context)

        config["server_round"] = server_round

        loss, _, return_dic = experiment.evaluate(parameters, config)

        return loss, return_dic

    return fn

partitioner = DirichletPartitioner(
            num_partitions=NUM_PARTITIONS,
            partition_by="label",
            alpha=0.3,
            seed=42,
        )
fds = FederatedDataset(
            dataset=DATASET,
            partitioners={"train": partitioner},
        )


if __name__ == '__main__':

    context = fl.init()

    fl.log(f"Flautim inicializado!!!")


    client_fn_callback = generate_client_fn(context)
    evaluate_fn_callback = evaluate_fn(context)
    server_fn_callback = generate_server_fn(context, eval_fn = evaluate_fn_callback)

    fl.log(f"Experimento criado!!!")

    run_federated(client_fn_callback, server_fn_callback, num_clients = NUM_PARTITIONS)

#### Referências
- [1] (Flautim tutoriais) Demais Tutorias disponíveis [aqui](https://github.com/FutureLab-DCC/flautim_tutoriais/tree/main)

- [2] (Artigo) Jee Cho, Y., Wang, J., and Joshi, G. (2022). Towards understanding biased client selection in federated learning. In Camps-Valls, G., Ruiz, F. J. R., and Valera, I., editors, Proceedings of The 25th International Conference on Artificial Intelligence and Statistics, volume 151 of Proceedings of Machine Learning Research, pages 10351–10375. PMLR.

- [3] (Discussão no Fórum do Flower - How do I write a custom client selection protocol? & "RepeatHalfSamplingFedAvg") https://discuss.flower.ai/t/how-do-i-write-a-custom-client-selection-protocol/74

- [4] (Discussão no Fórum do Flower - Custom client selection strategy) https://discuss.flower.ai/t/custom-client-selection-strategy/63



<p style="text-align: center;">
✨ Disponibilizado por Carla B. Ferreira, aluna de IC do FutureLab, em 2025. ✨
</p>
